In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using BedsData
using ForecastData
using GeographicData

In [4]:
using PatientAllocation

In [5]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI"]

start_date = Date(2020, 5, 1)
end_date   = Date(2020, 5, 30)

pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [6]:
N = length(states);
T = (end_date - start_date).value + 1;

In [7]:
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [8]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)[:];

In [9]:
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [10]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [11]:
model = patient_allocation(
    beds,
    forecast_initial,
    forecast_admitted,
    forecast_discharged,
    adj,
    hospitalized_days=hospitalized_days,
    send_new_only=true,
    sendrecieve_switch_time=0,
    min_send_amt=0,
    smoothness_penalty=0,
    setup_cost=0,
    sent_penalty=0,
    balancing_thresh=0.6,
    balancing_penalty=100.0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4244 rows, 3600 columns and 227820 nonzeros
Model fingerprint: 0xf917f33f
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-03, 1e+04]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 3057 rows and 1325 columns
Presolve time: 0.06s
Presolved: 1187 rows, 2275 columns, 136562 nonzeros

Ordering time: 0.02s

Barrier performed 0 iterations in 0.12 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 900 iterations and 0.12 seconds
Optimal objective  9.329002801e+03
termination status: OPTIMAL
solve time: 0.132s
objective function value: 9329.003


In [12]:
overflow_per_day = (i,t) -> sum(max.(0,
    forecast_initial[i] - sum(forecast_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t-1])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
    - beds[i])
)
overflow = i -> sum(overflow_per_day(i,t) for t=1:size(sent,3))
println("Total overflow: ", sum(overflow.(1:length(states))))

Total overflow: 8399.866149902344


In [13]:
total_patients = (i,t) -> (
    forecast_initial[i] - sum(forecast_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
);

In [14]:
load = (i,t) -> total_patients(i,t) / beds[i];

In [15]:
summary = DataFrame(
    state=states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    overflow=overflow.(1:length(states)),
    avg_load=[sum(load(i,t) for t in 1:T)/T for i in 1:N],
)

,state,total_sent,total_received,overflow,avg_load
,String,Float64,Float64,Float64,Float64
1,CT,1788.14,1185.5,769.123,0.503109
2,DE,190.847,173.343,0.0,0.390188
3,MA,921.328,838.15,0.0,0.438457
4,MD,2409.13,82.1432,181.623,0.437804
5,ME,22.6221,431.211,0.0,0.323306
6,NH,420.253,99.9529,0.0,0.45607
7,NJ,4368.36,3703.56,7449.12,0.570416
8,NY,47.6205,9648.48,0.0,0.454286
9,PA,5656.35,0.0,0.0,0.497542


In [16]:
println("Avg load:")
sum(sum(load(i,t) for t in 1:T)/T for i in 1:N)/N

Avg load:


0.4479631787094929

In [17]:
println("Avg load over threshold:")
sum(sum(max(0, load(i,t)-0.6) for t in 1:T)/T for i in 1:N)/N

Avg load over threshold:


0.030971237948944096

In [18]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(states))
insertcols!(sent_matrix, 1, :state => states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,68.7596,59.6378,0.0,304.852,95.2886,237.716,913.616,0.0,108.272
2,DE,22.6844,0.0,0.0,20.7423,0.0,0.0,39.6381,107.782,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,42.0897,0.0,96.4465,782.792,0.0,0.0
4,MD,0.0,2.88495,0.0,0.0,0.0,0.0,819.365,1586.88,0.0,0.0
5,ME,0.37695,0.0,8.408,0.0,0.0,4.66438,0.0,0.0,0.0,9.1728
6,NH,69.3104,0.0,223.784,0.0,4.1836,0.0,0.0,0.0,0.0,122.975
7,NJ,0.0,33.1664,484.97,0.0,0.0,0.0,0.0,3850.23,0.0,0.0
8,NY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.6205
9,PA,1065.03,68.5325,0.0,61.4009,0.0,0.0,2428.54,2032.85,0.0,0.0


In [19]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(sum(max.(0, forecast_admitted), dims=2)[:] - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(states));

In [20]:
outcomes = DataFrame()
for (i,s) in enumerate(states)
    single_state_outcome = DataFrame(
        state=fill(s, T),
        day=start_date .+ Dates.Day.(0:T-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        new_patients=forecast_admitted[i,:],
        total_patients=[total_patients(i,t) for t in 1:T],
        capacity=fill(beds[i], T),
        overflow=[overflow_per_day(i,t) for t in 1:T],
        load=[load(i,t) for t in 1:T],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:T],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:T],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("patient_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,new_patients,total_patients,capacity,overflow,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,CT,2020-05-01,321.51,0.0,321.51,2132.26,1840.25,613.52,1.15868,"[(""ME"", 272.982), (""NH"", 48.5282)]",[]
2,DE,2020-05-01,0.0,33.1664,34.1128,289.46,506.75,0.0,0.571208,[],"[(""NJ"", 33.1664)]"
3,MA,2020-05-01,0.0,225.41,327.558,2398.65,3997.75,0.0,0.6,[],"[(""NJ"", 225.41)]"
4,MD,2020-05-01,308.294,0.0,308.294,2102.08,2228.75,181.623,0.943165,"[(""NY"", 308.294)]",[]
5,ME,2020-05-01,0.0,344.114,4.4961,378.001,668.0,0.0,0.565869,[],"[(""CT"", 272.982), (""RI"", 71.132)]"
6,NH,2020-05-01,0.0,48.5282,32.9998,230.206,584.5,0.0,0.393851,[],"[(""CT"", 48.5282)]"
7,NJ,2020-05-01,922.172,0.0,922.172,7083.11,4553.25,3452.03,1.55562,"[(""DE"", 33.1664), (""MA"", 225.41), (""NY"", 663.596)]",[]
8,NY,2020-05-01,0.0,971.889,1051.65,8008.07,10406.2,0.0,0.769544,[],"[(""MD"", 308.294), (""NJ"", 663.596)]"
9,PA,2020-05-01,0.0,0.0,674.188,4603.58,7980.25,0.0,0.576872,[],[]


In [21]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,new_patients,total_patients,capacity,overflow,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,NJ,2020-05-01,922.172,0.0,922.172,7083.11,4553.25,3452.03,1.55562,"[(""DE"", 33.1664), (""MA"", 225.41), (""NY"", 663.596)]",[]
2,NJ,2020-05-02,915.841,0.0,915.841,6000.03,4553.25,2362.62,1.31775,"[(""MA"", 88.2), (""NY"", 827.641)]",[]
3,NJ,2020-05-03,910.815,0.0,910.815,4961.67,4553.25,1319.23,1.0897,"[(""MA"", 77.218), (""NY"", 833.596)]",[]
4,NJ,2020-05-04,913.432,0.0,913.432,3955.06,4553.25,315.24,0.868623,"[(""MA"", 60.0887), (""NY"", 853.344)]",[]
5,NJ,2020-05-05,194.78,0.0,940.136,3693.21,4553.25,0.0,0.811115,"[(""MA"", 34.0534), (""NY"", 160.727)]",[]
6,NJ,2020-05-06,0.0,0.0,938.541,3596.02,4553.25,0.0,0.78977,[],[]
7,NJ,2020-05-07,0.0,0.0,866.922,3365.71,4553.25,0.0,0.739188,[],[]
8,NJ,2020-05-08,0.0,0.0,844.234,3058.6,4553.25,0.0,0.67174,[],[]
9,NJ,2020-05-09,0.0,27.7211,816.379,2731.95,4553.25,0.0,0.6,[],"[(""MD"", 27.7211)]"


In [22]:
println("Sent to:")
Dict(states[i] => states[row] for (i,row) in enumerate(eachrow(sum(sent, dims=3)[:,:,1] .> 0)))

Sent to:


Dict{String,Array{String,1}} with 10 entries:
  "NH" => ["CT", "MA", "ME", "RI"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "RI"]
  "RI" => ["CT", "MA", "ME", "NJ", "NY"]
  "MA" => ["ME", "NJ", "NY"]
  "ME" => ["CT", "MA", "NH", "RI"]
  "NY" => ["RI"]
  "NJ" => ["DE", "MA", "NY"]
  "DE" => ["CT", "MD", "NJ", "NY"]
  "PA" => ["CT", "DE", "MD", "NJ", "NY"]
  "MD" => ["DE", "NJ", "NY"]